In [18]:
import pandas as pd
from pandas import DataFrame
import glob
import timeit

In [19]:
# fillna - used because null/None is float, not str, had to make it a str, for max/min
def get_inventory(file_type:str) -> DataFrame:
    if file_type == 'parquet':
        df = pd.read_parquet('./data/checkouts/inventory_parquet')
    elif 'csv' in file_type:
        df = (pd
          .read_csv("./data/checkouts/Library_Collection_Inventory.csv",
                    header=0, 
                    dtype={"BibNum":"int64", "Title":"str","Author":"str", "Subjects":"str"})
         )  
    df = df.drop_duplicates(subset='BibNum')
    return df

def get_inventory_no_dedup(file_type:str) -> DataFrame:
    if file_type == 'parquet':
        df = pd.read_parquet('./data/checkouts/inventory_parquet')
    elif 'csv' in file_type:
        df = (pd
          .read_csv("./data/checkouts/Library_Collection_Inventory.csv",
                    header=0, 
                    dtype={"BibNum":"int64", "Title":"str","Author":"str", "Subjects":"str"})
         )  
    return df
    

def get_checkouts(file_type:str) -> DataFrame:
    if file_type == 'multi-csv':
        li = []
        for filename in glob.glob("./data/checkouts/Checkouts_By_Title_Data_Lens_*.csv"):
            df = pd.read_csv(filename, index_col=None, header=0)
            li.append(df)

        df = pd.concat(li, axis=0, ignore_index=True)
    elif file_type =='single-csv':
        df = pd.read_csv("./data/checkouts/Checkouts.csv", 
                         header=0
                        )
    elif file_type =='parquet':
        df = pd.read_parquet('./data/checkouts/checkouts_parquet')  
    return df

def writter_csv(df: DataFrame) -> None:
    if single_file:
        df.to_csv('./output/output.csv', mode="w+")
         
def writter_parquet(df: DataFrame) ->None:
    df.to_parquet("./output/parquet_output")
    
def add_formatted_checkout(file_type:str)->DataFrame:
    df = get_checkouts(file_type)
    df['CheckoutTime_formated']=pd.to_datetime(df.CheckoutDateTime, format="%m/%d/%Y %I:%M:%S %p")
    return df

In [20]:
file_types = ["parquet", "multi-csv", "single-csv"]
wr = [writter_csv, writter_parquet]

Reading files

In [21]:
%timeit -n 1 -r 1 get_checkouts("parquet")
%timeit -n 1 -r 1 get_checkouts("multi-csv")
%timeit -n 1 -r 1 get_checkouts("single-csv")

2min 43s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/timeit.py:204: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  t = self.timeit(number)


2min 57s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/timeit.py:204: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  t = self.timeit(number)


4min 30s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Getting head of read file

In [9]:
%timeit -n 1 -r 5 get_checkouts("parquet").head()
%timeit -n 1 -r 5 get_checkouts("multi-csv").head()
%timeit -n 1 -r 5 get_checkouts("single-csv").head()

1.11 s ± 37.5 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
1.54 s ± 15.8 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
1.45 s ± 17.4 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


Reading small file and writting to file

In [7]:
for file_type in file_types:
    for writter in wr:
        print(f"file_type: {file_type}, writter: {writter.__name__}")
        %timeit -n 1 -r 5 writter(get_inventory_no_dedup(file_type))

compute: threads, file_type: parquet, writter: writter_csv, single_file = False
18.1 s ± 486 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
compute: threads, file_type: parquet, writter: writter_parquet, single_file = False
14.6 s ± 118 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
compute: threads, file_type: parquet, writter: writter_csv, single_file = True
17.8 s ± 220 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
compute: threads, file_type: multi-csv, writter: writter_csv, single_file = False
19.1 s ± 193 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
compute: threads, file_type: multi-csv, writter: writter_parquet, single_file = False
15.5 s ± 51.2 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
compute: threads, file_type: multi-csv, writter: writter_csv, single_file = True
18.9 s ± 65.4 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
compute: threads, file_type: single-csv, writter: writter_csv, single_file = False
19 s ± 47.7 ms per lo

KeyboardInterrupt: 

Read no dedup, filter, write

In [ ]:
for file_type in file_types:
    for writter in wr:
        print(f"file_type: {file_type}, writter: {writter.__name__}")
        stop_list = []
        for i in range(5):
            start = timeit.default_timer()
            df = get_inventory_no_dedup(file_type)[])
            df = df[df["ReportDate"]=='10/01/2017']]
            stop = timeit.default_timer()
            stop_list = stop_list.append(stop-start)
        print(f"file_type: {file_type}, runtime: {sum(stop_list)/len(stop_list)}")
        %timeit -n 1 -r 5 

Reading file, dropping duplicates writting to file

In [ ]:
   for file_type in file_types:
        for writter in wr:
            print(f"file_type: {file_type}, writter: {writter.__name__}, single_file = {writter[1]}")
            %timeit -n 1 -r 5 writter(get_inventory(file_type))

Reading Big file and writting to file

In [ ]:
for file_type in file_types:
    for writter in wr:
        print(f"file_type: {file_type}, writter: {writter.__name__}")
        %timeit -n 1 -r 5 writter(get_checkouts(file_type))

Read + convert to datetime and write


In [ ]:
for compute in computes:
    for file_type in file_types:
        for writter in wr:
            print(f"file_type: {file_type}, writter: {writter.__name__}")
            %timeit -n 1 -r 5 writter(add_formatted_checkout(file_type))

Read convert to datetime, get min datetime value

In [ ]:
for file_type in file_types:
    stop_list = []
    for i in range(5):
        start = timeit.default_timer()
        df = add_formatted_checkout(file_type)
        df.CheckoutTime_formated.min()
        stop = timeit.default_timer()
        stop_list = stop_list.append(stop-start)
    print(f"file_type: {file_type}, runtime: {sum(stop_list)/len(stop_list)}")

Inner Join and head

In [ ]:
for file_type in file_types:
    stop_list = []
    for i in range(5):
        start = timeit.default_timer()
        checkouts = get_checkouts("parquet")[["BibNumber", "CheckoutDateTime"]]
        inventory = get_inventory("parquet")[["BibNum", "Author"]]
        merged = pd.merge(checkouts, inventory, left_on="BibNumber", right_on="BibNum", how="inner")
        merged.head()
        stop = timeit.default_timer()
        stop_list = stop_list.append(stop-start)
    print(f"file_type: {file_type}, runtime: {sum(stop_list)/len(stop_list)}")

Anti join and head

In [8]:
for file_type in file_types:
    stop_list = []
    for i in range(5):
        start = timeit.default_timer()
        checkouts = get_checkouts("parquet")[["BibNumber", "CheckoutDateTime"]]
        inventory = get_inventory("parquet")[["BibNum", "Author"]]
        merged = pd.merge(checkouts, inventory, left_on="BibNumber", right_on="BibNum", how="outer", indicator=True)
        merged[merged._merge=="right_only"].head()
        stop = timeit.default_timer()
        stop_list = stop_list.append(stop-start)
    print(f"file_type: {file_type}, runtime: {sum(stop_list)/len(stop_list)}")

,BibNumber,CheckoutDateTime,BibNum,Author,_merge
875397,NaN,NaN,1847748.0,None,right_only
875398,NaN,NaN,214902.0,United States. National Resources Committee. S...,right_only
875399,NaN,NaN,361929.0,"Wetmore, Alexander, 1886-1978.",right_only
875400,NaN,NaN,2071549.0,"Carroll, Lenore, 1939-",right_only
875401,NaN,NaN,306179.0,"Raymond, William Galt, 1859-1926.",right_only


Read + Row count

In [ ]:
for compute in computes:
    for file_type in file_types:
        print(f"file_type: {file_type}")
        %timeit -n 1 -r 5 get_checkouts(file_type).count().compute()

Read + explode + write

In [ ]:
for file_type in file_types:
    for writter in wr:
        stop_list = []
        for i in range(5):
            start = timeit.default_timer()
            inventory = get_inventory(file_type)[["BibNum", "Subjects"]]
            inventory['Subject'] = inventory['Subjects'].str.split(',').fillna("")
            inventory = inventory.explode("Subject")
            writter(inventory)
            stop = timeit.default_timer()
            stop_list = stop_list.append(stop-start)
        print(f"file_type: {file_type}, writter: {writter.__name__}, runtime: {sum(stop_list)/len(stop_list)}")

Read+explode+dcount


In [ ]:
for file_type in file_types:
    stop_list = []
    for i in range(5):
        start = timeit.default_timer()
        inventory = get_inventory(file_type)[["BibNum", "Subjects"]]
        inventory['Subject'] = inventory['Subjects'].str.split(',').fillna("")
        inventory = inventory.explode("Subject")
        inventory = inventory.groupby(inventory.Subject)["BibNum"].nunique())
        inventory.head()
        stop = timeit.default_timer()
        stop_list = stop_list.append(stop-start)
    print(f"file_type: {file_type}, runtime: {sum(stop_list)/len(stop_list)}")

Read + explode + join + dcount + head

In [ ]:
for file_type in file_types:
    stop_list = []
    for i in range(5):
        start = timeit.default_timer()
        checkouts = get_checkouts("parquet")[["BibNumber", "CheckoutDateTime"]]
        inventory = get_inventory(file_type)[["BibNum", "Subjects"]]
        inventory['Subject'] = inventory['Subjects'].str.split(',').fillna("")
        inventory = inventory.explode("Subject")
        merged = pd.merge(checkouts, inventory, left_on="BibNumber", right_on="BibNum", how="inner")
        merged = merged.groupby(merged.index)["BibNum"].nunique().astype("int64"))
        merged.head()
        stop = timeit.default_timer()
        stop_list = stop_list.append(stop-start)
    print(f"file_type: {file_type}, runtime: {sum(stop_list)/len(stop_list)}")

Read + explode + join + dcount + top 10 by dist count

In [ ]:
for file_type in file_types:
    stop_list = []
    for i in range(1):
        start = timeit.default_timer()
        checkouts = get_checkouts("parquet")[["BibNumber", "CheckoutDateTime"]]
        inventory = get_inventory(file_type)[["BibNum", "Subjects"]]
        inventory['Subject'] = inventory['Subjects'].str.split(',').fillna("")
        inventory = inventory.explode("Subject")
        merged = pd.merge(checkouts, inventory, left_on="BibNumber", right_on="BibNum", how="inner")
        merged = merged.groupby(merged.Subject)["BibNum"].nunique().astype("int64")).nlargest(10)
        merged.head(10)
        stop = timeit.default_timer()
        stop_list = stop_list.append(stop-start)
    print(f"file_type: {file_type}, runtime: {sum(stop_list)/len(stop_list)}")

Read+convert to datetime + get first 10 checkouts

In [15]:
for compute in computes:
    for file_type in file_types:
        print(f"file_type: {file_type}")
        %timeit -n 1 -r 5 add_formatted_checkout(file_type).nsmallest(10, "CheckoutTime_formated").head(10)

compute: threads, file_type: parquet, writter: writter_csv, single_file = False



KeyboardInterrupt

